In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import glob
import requests
# from geoalchemy2 import Geometry, WKTElement

import psycopg2  # (if it is postgres/postgis)
conn = psycopg2.connect(database="postgres", user="wenfeixu", password="",
    host="localhost")
cursor = conn.cursor()


from sqlalchemy import create_engine
engine = create_engine('postgresql://wenfeixu:@localhost:5432/postgres')
    
from geoalchemy2 import Geometry, WKTElement



# Goal of the Notebook
1. For 1940s tract boundaries: find the dominant HOLC label
1. For all tracts across all years: retain those that intersect with HOLC boundaries
2. For all tracts across all years: Label each year's tract based on their dominant HOLC grade

# Datasets Used

- HOLC polygons from University of Richmond Digital Scholarship Lab
- Historical Census Data from IPUMS National Historical GIS database
****

# 0. Get Data

1940s tract boundaries and data

In [ ]:
sql= '''
    select * from us_tract_1940_conflated_4326
    '''
us_tract_1940_conflated_4326 = gpd.GeoDataFrame.from_postgis(sql, conn,geom_col='geom',crs={'init':'epsg:4326'} )
us_tract_1940_conflated_4326['tract_area'] = us_tract_1940_conflated_4326.to_crs({'init':'epsg:3857'}).area

In [ ]:
us_tract_1940_conflated_4326.shape

## 0.2 HOLC Boundaries and Labels

In [ ]:
sql= '''
    select * from holc_all_dump_4326
    '''
holc = gpd.GeoDataFrame.from_postgis(sql, conn,geom_col='geometry',crs={'init':'epsg:4326'} )
holc['holc_area'] = holc.to_crs({'init':'epsg:3857'}).area

holc['HOLC_Grade']  = holc['HOLC_Grade'].replace({'Green':'A','Blue':'B','Yellow':'C','Red':'D'})

FHA Boundaries and Labels

In [ ]:
fha = gpd.read_file('../data/FHA Map/FHA Map.shp')
fha['id']=fha.index.values
fha = fha[fha['geometry'].isna()==False]
fha =fha.rename(columns={'grade':'fha_grade'})
fha['fha_area'] = fha.to_crs({'init':'epsg:3857'}).area

fha1= fha.copy()
fha1['geom'] = fha1['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
fha1 = fha1.drop(columns=['geometry'])
fha1.to_sql('fha_chicago', engine, if_exists='replace', index=False, 
                         dtype={'geom': Geometry('Polygon', srid= 4326)})

All Census data

In [ ]:
sql = 'select * from us_tract_1930_conflated_4326'
df_1930 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
sql = 'select * from us_tract_1940_conflated_4326'
df_1940 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
sql = 'select * from us_tract_1950_conflated_4326'
df_1950 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
sql = 'select * from us_tract_1960_conflated_4326'
df_1960 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
sql = 'select * from us_tract_1970_conflated_4326'
df_1970 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
sql = 'select * from us_tract_1980_conflated_4326'
df_1980 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
sql = 'select * from us_tract_1990_conflated_4326'
df_1990 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
sql = 'select * from us_tract_2000_conflated_4326'
df_2000 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
sql = 'select * from us_tract_2010_conflated_4326'
df_2010 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )


Optional read a version already intersected with HOLC and reducing total tracts to crosswalk later

In [ ]:
# sql = 'select a.* from us_tract_1930_conflated_4326 a, holc_updated_4326 b where st_intersects(a.geom,b.geom)'
# df_1930 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
# sql = 'select a.* from us_tract_1940_conflated_4326 a, holc_updated_4326 b where st_intersects(a.geom,b.geom)'
# df_1940 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
# sql = 'select a.* from us_tract_1950_conflated_4326 a, holc_updated_4326 b where st_intersects(a.geom,b.geom)'
# df_1950 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
# sql = 'select a.* from us_tract_1960_conflated_4326 a, holc_updated_4326 b where st_intersects(a.geom,b.geom)'
# df_1960 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
# sql = 'select a.* from us_tract_1970_conflated_4326 a, holc_updated_4326 b where st_intersects(a.geom,b.geom)'
# df_1970 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
# sql = 'select a.* from us_tract_1980_conflated_4326 a, holc_updated_4326 b where st_intersects(a.geom,b.geom)'
# df_1980 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
# sql = 'select a.* from us_tract_1990_conflated_4326 a, holc_updated_4326 b where st_intersects(a.geom,b.geom)'
# df_1990 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
# sql = 'select a.* from us_tract_2000_conflated_4326 a, holc_updated_4326 b where st_intersects(a.geom,b.geom)'
# df_2000 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )
# sql = 'select a.* from us_tract_2010_conflated_4326 a, holc_updated_4326 b where st_intersects(a.geom,b.geom)'
# df_2010 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom' )


# 1. 1940s Tracts - HOLC Labels

### 1.1 Create an overlay btwn 1940 Tracts and HOLC
- Find all the "pieces" of tracts that overlap with an HOLC area
- Find the biggest piece for each tract
- If this piece has more than 25% tract coverage, it will be the main grade

In [ ]:
us_tract_1940_pieces = gpd.overlay(us_tract_1940_conflated_4326[['geom', 'id', 'gisjoin','tract_area']], holc, how='intersection')

us_tract_1940_pieces = us_tract_1940_pieces[us_tract_1940_pieces.HOLC_Grade.isin(['C', 'D', 'B', 'A', 'E', 'Yellow', 'Green', 'Blue','Red'])]
us_tract_1940_pieces['perc_tract'] = us_tract_1940_pieces.to_crs({'init':'epsg:3857'}).area/us_tract_1940_pieces['tract_area']
us_tract_1940_pieces['perc_holc'] = us_tract_1940_pieces.to_crs({'init':'epsg:3857'}).area/us_tract_1940_pieces['holc_area']

### 1.2 For each tract we want to find the dominant group
For each tract, we want to keep the label that has the largest percentage overlap with the trac. 

In [ ]:
us_tract_1940_pieces_tracts  =us_tract_1940_pieces.sort_values(['gisjoin','perc_tract']).groupby('gisjoin').tail(1)
us_tract_1940_pieces_holc  =us_tract_1940_pieces.sort_values(['gisjoin','perc_holc']).groupby('gisjoin').tail(1)

In [ ]:
us_tract_1940_pieces1 = us_tract_1940_pieces.sort_values(['gisjoin','perc_tract'],ascending=False).groupby('gisjoin').first().reset_index()

In [ ]:
us_tract_1940_pieces1.shape

### 1.3 Final Tract Labeling
We only want to keep those tracts that have at least 25% overlap

In [ ]:
tract_thres = 0.25
us_tract_1940_holc_final = us_tract_1940_conflated_4326[us_tract_1940_conflated_4326.gisjoin.isin(
                                        us_tract_1940_pieces1[us_tract_1940_pieces1['perc_tract']>tract_thres].gisjoin)].merge(us_tract_1940_pieces1[['gisjoin','map_id','HOLC_ID','map_name','HOLC_Grade','HOLC_Lette','name']])

### 1.4 Export to postgis

In [ ]:
us_tract_1940_holc_final['geom'] = us_tract_1940_holc_final['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))

us_tract_1940_holc_final.to_sql('us_tract_1940_holc_final', engine, if_exists='replace', index=False, 
                         dtype={'geom': Geometry('MultiPolygon', srid= 4326)})

# 2. 1940s Tracts - FHA Labels
This only applies for Chicago for now. 

In [ ]:
us_tract_1940_pieces_fha = gpd.overlay(us_tract_1940_conflated_4326[['geom', 'id', 'gisjoin','tract_area']], fha, how='intersection')

us_tract_1940_pieces_fha['perc_tract'] = us_tract_1940_pieces_fha.to_crs({'init':'epsg:3857'}).area/us_tract_1940_pieces_fha['tract_area']
us_tract_1940_pieces_fha['perc_fha'] = us_tract_1940_pieces_fha.to_crs({'init':'epsg:3857'}).area/us_tract_1940_pieces_fha['fha_area']


us_tract_1940_pieces_tracts_fha  =us_tract_1940_pieces_fha.sort_values(['gisjoin','perc_tract']).groupby('gisjoin').tail(1)
us_tract_1940_pieces_fha1  =us_tract_1940_pieces_tracts_fha.sort_values(['gisjoin','perc_fha']).groupby('gisjoin').tail(1)
us_tract_1940_pieces1_fha = us_tract_1940_pieces_fha1.sort_values(['gisjoin','perc_tract'],ascending=False).groupby('gisjoin').first().reset_index()

In [ ]:
tract_thres = 0.25
us_tract_1940_fha_final = us_tract_1940_conflated_4326[us_tract_1940_conflated_4326.gisjoin.isin(
                                        us_tract_1940_pieces1_fha[us_tract_1940_pieces1_fha['perc_tract']>tract_thres].gisjoin)].merge(us_tract_1940_pieces1_fha[['gisjoin','id_2','fha_area','fha_grade','perc_fha']])

In [ ]:
us_tract_1940_fha_final['geom'] = us_tract_1940_fha_final['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))

us_tract_1940_fha_final.to_sql('us_tract_1940_fha_final', engine, if_exists='replace', index=False, 
                         dtype={'geom': Geometry('MultiPolygon', srid= 4326)})

# 3. Create a crosswalk back to 1940s census tracts and weighted census values
Because we want to compare each decade's tract data to each other, we need to create a decade by decade cross walk back to 1940s. Using 1940s census tract levels so that we are mostly aggregating the data rather than splitting it up. Ultimately, we want a list of corresponding tract numbers for each 1940 tract. This can be one or many tracts. We're going to assume a population weighted sum for all the cross-walks.


This will involve: 
- Doing a spatial join between decade YYYY and census tract from 1940
- For each 1940 tract, we want to create a set of weights for other census tracts that intersect with the 1940s tract: 
    $$w_{i_{j,t}} = \frac{a_{i_{j,t} \cap t={1940}}}{a_{i_{j,t}}}$$ for 1940s tract $i$ and tract pieces $j$ if $\frac{a_{i_{j,t} \cap t={1940}}}{a_{i_{j,t}}}>.01$ In other words, the overlap has to be at least 1%
- Find the population contribution that each region makes to the HOLC zone $$p_{i_{j,t}}w_{i_{j,t}}$$
- Find the weighted average of each zone based on population  $$ \frac{x_{i_{j,t}} p_{i_{j,t}}w_{i_{j,t}}}{\sum_j p_{i_{j,t}}w_{i_{j,t}} }$$ 


In [ ]:
# features=['population','white','white_perc','black','black_perc','other',
#  'other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
#  'median_income_adj','median_income','hispanic_perc','hispanic','unemployed','unemployed_perc']

def get_weighted_values(year1,year2, perc=0.01):
    ## 2016 ACS data uses 2010 Census boundaries, though 2016 data not ultimately used in analysis. 
    
    sql ='''
    
    select * from (select x.*, y.gisjoin_1940, y.geom as geom_using,
    st_area(st_intersection(x.geom,y.geom))/st_area(y.geom)::decimal as tract_1940_perc,
    st_area(st_intersection(x.geom,y.geom))/st_area(x.geom)::decimal as tract_perc
    from
    (select distinct a.*,b.geom from (select distinct * from census_{} as a) as a, (select distinct * from us_tract_{}_conflated_4326 as b)  as b 
    where a."GISJOIN" = b.gisjoin) as x, 
    (select distinct gisjoin as gisjoin_1940, geom from (
        select distinct * from us_tract_1940_conflated_4326) as y
        ) as y
    where st_intersects(x.geom, y.geom)) as t where tract_perc >.01
    '''.format(year1,year2)
  
        
    ### Get all the census units who's area overlap is greater than the min threshold
    df_overlay_1 = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geom_using' )
    print(df_overlay_1.shape)
    
    ### Create a df that groups by the 1940s tracts
    df_overlay=df_overlay_1.groupby(['gisjoin_1940']).count()[['index']]
    df_overlay=df_overlay.rename(columns={'index':'count'})
    
    df_overlay_2 = df_overlay_1[df_overlay_1['tract_1940_perc']>perc]
    for each in features[year1]: 

        ### for the other features, use a population-weighted average
        weighted_sum = df_overlay_2.groupby(['gisjoin_1940']).apply(lambda x: np.sum(x[each]*x['population']*x['tract_perc'])/ np.sum(x['population']*x['tract_perc']))
        df_overlay[each]=weighted_sum
    df_overlay=df_overlay.fillna(0)
    print(df_overlay.shape)
    return df_overlay.reset_index(),df_overlay_1.reset_index()

In [ ]:
df_1940 = gpd.GeoDataFrame.from_postgis("select distinct * from us_tract_1940_conflated_4326", conn,crs={'init':'epsg:4326'}, geom_col='geom' )

def merge_overlay_geom(df,name):
    df_merge = df_1940[['gisjoin','geom']].merge(df.reset_index(),
                   left_on='gisjoin',
                   right_on='gisjoin_1940')

    #### Create the population density meausure
    #### Update: Going to convert to geography and gonna do this in postgis 
    df_merge['population_density']=df_merge['population']/df_merge.geometry.to_crs({'init': 'epsg:3857'}).area
    df_merge['geom'] = df_merge['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))
    df_merge.to_sql('{}'.format(name), engine, if_exists='replace', index=False, 
                             dtype={'geom': Geometry('MultiPolygon', srid= 4326)})



#### Features using

In [ ]:
features = {1930:['population','white','white_perc','black','black_perc','other','other_perc','median_homevalue','median_homevalue_adj','owned_perc'],
            1940:['population','white','white_perc','black','black_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
                  'owned_perc'],
            1950:['population','white','white_perc','black','black_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','owned_perc'],
            1960:['population','white','white_perc','black','black_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','owned_perc'],
            1970:['population','white','white_perc','black','black_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','unemployed_perc','owned_perc'],
            1980:['population','white','white_perc','black','black_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','unemployed','unemployed_perc','owned_perc'],
            1990:['population','white','white_perc','black','black_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','unemployed','unemployed_perc','owned_perc'],
            2000:['population','white','white_perc','black','black_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','unemployed','unemployed_perc','owned_perc'],
            2010:['population','white','white_perc','black','black_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
 'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','asian_perc','asian','unemployed','unemployed','unemployed_perc','owned_perc'],
#             2016:['population','white','white_perc','black','black_perc','other','other_perc','median_homevalue','median_homevalue_adj','homes','college','college_perc',
#  'median_income_adj','median_income','hispanic_perc','hispanic','asian_perc','asian','unemployed','unemployed_perc','owned_perc']}

In [ ]:
for y in list(features.keys()):
    print (y)
    if y !=2016:
        y1 = y
        y2 = y   
    else: 
        y1 = y
        y2= 2010
    df,df_1 = get_weighted_values(y1,y2)
    print(df.shape)
    merge_overlay_geom(df,'census_1940_overlay_{}_new'.format(y1))

# 4. Concat all the Census data and save as table
Here reloading to doublecheck

In [ ]:
years = [1930,1940,1950,1960,1970,1980,1990,2000,2010,2016]

Get data for all the years

In [ ]:
def get_crosswalked_data(year):

    sql= f'''
    select * from census_1940_overlay_{year}_new
    '''
    df = gpd.GeoDataFrame.from_postgis(sql, conn,geom_col='geom',crs={'init':'epsg:4326'} )
    return df

df = pd.concat([get_crosswalked_data(y) for y in years])
df=df.fillna(0)


HOLC

In [ ]:
t =  df[df.gisjoin_1940.isin(us_tract_1940_holc_final.gisjoin)]
t = df.merge(us_tract_1940_holc_final[['gisjoin','map_id', 'HOLC_ID', 'map_name', 'HOLC_Grade',
       'HOLC_Lette', 'name']],on='gisjoin')

t.to_postgis('all_tracts_holc_crosswalked_FINAL', engine, if_exists='replace')



FHA

In [ ]:
f =  df[df.gisjoin_1940.isin(us_tract_1940_fha_final.gisjoin)]
f= df.merge(us_tract_1940_fha_final[['gisjoin','id_2', 'fha_area', 'fha_grade', 'perc_fha']],on='gisjoin')

f.to_postgis('all_tracts_fha_crosswalked_FINAL', engine, if_exists='replace')
